<a target="_blank" href="https://colab.research.google.com/github/kircherlab/MPRAlib/examples/mpralib.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
!pip --quiet install "MPRAlib==0.5.0"

# Loading Data and Understanding the MPRAlib data structure

Here we will load a barcode count file (output of [MPRAsnakeflow](https://github.com/kircherlab/MPRAsnakeflow)) into MPRAlib, more precise into an `MPRAData` data object in `mpralib.mpradata`. MPRAlib will generate an [AnnData](https://anndata.readthedocs.io) object out of it, which is acessable via `MRAData.data`:

<img src="https://raw.githubusercontent.com/scverse/anndata/main/docs/_static/img/anndata_schema.svg" alt="AnnData" width="480 "/>

## Core data

We defined two objects inhereted from `MPRAdata`:
- `MPRABarcodeData` for DNA and RNA counts per barcode
- `MPRAOligoData` for RNA and DNA counts aggregated per oligo

`MPRABarcodeData` can be generated from `MPRABarcodeData` but not vice versa. So we start with the `MPRABarcodeData` object:

### `MPRABarcodeData`

- `var`: barcodes
- `obs`: replicates

Oligo names are also stored in `var` like `MPRABarcodeData.data.var["oligo"]` and can also be accessed via `MPRABarcodeData.oligos`. Further DNA and RNA counts are stored in `layers` like `MPRABarcodeData.data.layers["dna"]` and `layers["rna"]`. This are the raw counts and those layers are never been modified. You can also access this data via `MPRABarcodeData.raw_rna_counts` and `MPRABarcodeData.raw_dna_counts`.

Metadata is stored in the `MPRABarcodeData.data.uns` dictionary.

#### Extended count data

There are some further layers that are dynamically updated via the library. This are:

1. Normalization layers in `MPRABarcodeData.data.layers["dna_normalized"]` and `MPRABarcodeData.data.layers["rna_normalized"]`. Usually we have to normalize the data to account for different sequencing depths. This is done by dividing the raw counts by the sum of the counts all counts per replicate and scale them (usually `1e6` like counts per million). We use a pseudo count of `1` to avoid division by zero. So barcodes are allowed to be not observed in either RNA or DNA. When both counts are zero the library will take this barcodes as not observed and will not use it for counting/normalization etc. The pseudocount can be modified via `MPRABarcodeData.PSEUDOCOUNT` e.g. avoiding it via `MPRABarcodeData.PSEUDOCOUNT = 0`.
2. Sampling layers in `MPRABarcodeData.data.layers["dna_sampling"]` and `MPRABarcodeData.data.layers["rna_sampling"]`. This is only an edge case when you want to downsample your counts.

The library usually generates the layers and takes care of them. E.g. when barcode filtering is applied, the normalization layers are updated accordingly.

MPRAlib provides some getters (in python properties) to access the data intuitively. E.g. `MPRABarcodeData.raw_dna_counts` will return the raw DNA counts. `MPRABarcodeData.raw_rna_counts` for RNA accordingly. If you use the property `MPRABarcodeData.rna_counts` or `MPRABarcodeData.dna_counts` it will return the latest counts. That can be the raw counts (if nothing was done), filtered counts when where some barcode counts are set to zero according to the barcode filter mask (see below), sampled or sampled and filtered counts. If you want to get norlazied counts use `MPRABarcodeData.normalized_rna_counts` or `MPRABarcodeData.normalized_dna_counts`. It returns the layer `"rna_normalized"` or `"dna_normalized"` which is a normalization of `MPRABarcodeData.rna_counts` or `MPRABarcodeData.dna_counts`.

#### Barcode filtering

Barcode filters are (or in AnnData filter for `var`) stored in a n_barcodes x n_replicates matrix within the AnnData object `MPRABarcodeData.data.varm["var_filter"]`. You can set any new filter using the setter `MPRABarcodeData.var_filter = new_filter`. The filter is a boolean matrix with `True` for barcodes that should be removed and `False` for barcodes that should be kept. This setter magically also updates the normalized counts for you.

There are pre implemented methods to filter barcodes, like detecting outliers. We will cover this part later.

### Grouping Data by Oligo - The `MPRAOligoData` object

Usually when we work with MPRA data we are not interested by the barcode count itself but an aggregated count version per oligo. This is done by grouping the data by oligo and writing a new MPRAlib object with a new AnnData object, called `MPRAOligoData` and `MPRAOligoData.data` accordingly. `MPRAOligoData` also inherits from `MPRAData` so has very similar functionality than `MPRABarcodeData`. It is a new AnnData object because the data structure is slightly different. The `var` are now the oligos and the `obs` remain the replicates. The layers `"dna"`, `"rna"`, `"dna_normalized"`, and  `"rna_normalized"` exist also for the grouped data but are now aggregated per oligo. 

When generating a `MPRAOligoData` from an `MPRABarcodeData` usually the filtered counts or sampled counts are used for aggregation (if applicable). Also you can define a barcode threshold to filter out oligos that have not enough barcodes. This threshold can be set by `MPRAdata.barcode_threshold = 10` for example. The default is 1 (no oligo is removed). Barcode thereshold is only applied when running correlations on the `MPRAOligoData` object or saving outputs. Otherwise you will see all counts from all oligos.

For this purpose we have one additional layers: `MPRAOligoData.data.layers["barcode_counts"]` counts the number of barcodes per oligo that was used for the aggregation (or simple property `MPRAOligoData.barcode_counts`). The layer is also available in MPRABarcode data and shows the same value for each barcode within the same oligo.

Maybe the most interesting layer is the one for activity (whch also exists in the barcode object but there this can be very sparse data): `MPRAOligoData.data.layers["activity"]` stores the log2 rna/dna ratio and can also be accessed via `MPRAOligoData.activities`.

## Example

Let's start with loading a barcode count file from [MPRAsnakeflow](https://github.com/kircherlab/MPRAsnakeflow):

In [13]:
# Loading the MPRAlib library
from mpralib.mpradata import MPRABarcodeData
# Loading other libraries
import pandas as pd
import numpy as np

# Load the data
mpra_barcode_data = MPRABarcodeData.from_file("../resources/reporter_experiment.barcode.HEPG2.fromFile.default.all.tsv.gz")

# Getting counts, no filtering/sampling done, so raw counts
print("DNA counts")
display(pd.DataFrame(mpra_barcode_data.dna_counts[:,0:5], index=mpra_barcode_data.obs_names, columns=mpra_barcode_data.var_names[0:5]))
print("RNA counts")
display(pd.DataFrame(mpra_barcode_data.rna_counts[:,0:5], index=mpra_barcode_data.obs_names, columns=mpra_barcode_data.var_names[0:5]))


DNA counts


barcode,TACTCTCCGTGCCCA,GGTATAACATCTCCG,TTAGGAGTCACACGT,GAATATAACACCCGA,AAACACCGCGCTCTA
1,1,0,0,1,0
2,0,3,1,1,0
3,0,1,1,1,1


RNA counts


barcode,TACTCTCCGTGCCCA,GGTATAACATCTCCG,TTAGGAGTCACACGT,GAATATAACACCCGA,AAACACCGCGCTCTA
1,1,0,0,1,0
2,0,1,1,1,0
3,0,2,2,3,1


### Correlation

We can correlate replicates based on DNA/RNA counts or activity. We are able to do this on a barcode level. But usually this generates very very low correlations:

In [14]:
print("Pairwise Pearson correlation activity")
print(pd.DataFrame(mpra_barcode_data.correlation(), index=mpra_barcode_data.obs_names, columns=mpra_barcode_data.obs_names).round(3))
print("Pairwise Pearson correlation normalized DNA")
print(pd.DataFrame(mpra_barcode_data.correlation(count_type="dna"), index=mpra_barcode_data.obs_names, columns=mpra_barcode_data.obs_names).round(3))
print("Pairwise Pearson correlation normalized RNA")
print(pd.DataFrame(mpra_barcode_data.correlation(count_type="rna"), index=mpra_barcode_data.obs_names, columns=mpra_barcode_data.obs_names).round(3))

Pairwise Pearson correlation activity
       1      2      3
1  1.000  0.018 -0.004
2  0.018  1.000  0.137
3 -0.004  0.137  1.000
Pairwise Pearson correlation normalized DNA
       1      2      3
1  1.000 -0.121 -0.123
2 -0.121  1.000 -0.128
3 -0.123 -0.128  1.000
Pairwise Pearson correlation normalized RNA
       1      2      3
1  1.000 -0.057 -0.053
2 -0.057  1.000 -0.065
3 -0.053 -0.065  1.000


So let#s tdo this for aggregated data (per Oligo not per barcode):

In [15]:
mpra_oligo_data = mpra_barcode_data.oligo_data

print("Pairwise Pearson correlation")
print(pd.DataFrame(mpra_oligo_data.correlation(), index=mpra_oligo_data.obs_names, columns=mpra_oligo_data.obs_names).round(3))
print("Parwise Spearman correlation")
print(pd.DataFrame(mpra_oligo_data.correlation("spearman", "activity"), index=mpra_oligo_data.obs_names, columns=mpra_oligo_data.obs_names).round(3))


# Setting a different barcode threshold 1, 10, 50 and getting the average across replicates
for threshold in [1, 10, 50]:
    mpra_oligo_data.barcode_threshold = threshold
    print(f"Mean Pearson correlation activity, BC threshold {mpra_oligo_data.barcode_threshold}: {mpra_oligo_data.correlation().flatten()[[1,2,5]].mean().round(3)}")
    
    print(f"Mean Pearson correlation RNA counts (normalized), BC threshold {mpra_oligo_data.barcode_threshold}: {mpra_oligo_data.correlation(count_type="rna").flatten()[[1,2,5]].mean().round(3)}")
    
    print(f"Mean Pearson correlation DNA counts (normalized), BC threshold {mpra_oligo_data.barcode_threshold}: {mpra_oligo_data.correlation(count_type="dna").flatten()[[1,2,5]].mean().round(3)}")

    print("")

Pairwise Pearson correlation
       1      2      3
1  1.000  0.382  0.412
2  0.382  1.000  0.417
3  0.412  0.417  1.000
Parwise Spearman correlation
       1      2      3
1  1.000  0.345  0.374
2  0.345  1.000  0.350
3  0.374  0.350  1.000
Mean Pearson correlation activity, BC threshold 1: 0.404
Mean Pearson correlation RNA counts (normalized), BC threshold 1: 0.608
Mean Pearson correlation DNA counts (normalized), BC threshold 1: 0.29

Mean Pearson correlation activity, BC threshold 10: 0.629
Mean Pearson correlation RNA counts (normalized), BC threshold 10: 0.763
Mean Pearson correlation DNA counts (normalized), BC threshold 10: 0.464

Mean Pearson correlation activity, BC threshold 50: 0.839
Mean Pearson correlation RNA counts (normalized), BC threshold 50: 0.886
Mean Pearson correlation DNA counts (normalized), BC threshold 50: 0.638



Using such strict barcode threshold will also reduce our oligos available per replicate:

In [16]:
for threshold in [1, 10, 50]:
    mpra_oligo_data.barcode_threshold = threshold
    print(f"Number of oligos per individual replicate, using BC threshold {mpra_oligo_data.barcode_threshold}")
    print(np.sum(mpra_oligo_data.barcode_counts >= mpra_oligo_data.barcode_threshold,axis=1))


Number of oligos per individual replicate, using BC threshold 1
[2243 2250 2253]
Number of oligos per individual replicate, using BC threshold 10
[1359 1349 1357]
Number of oligos per individual replicate, using BC threshold 50
[148 146 154]


## Barcode filtering and outlier detection

There is some pre-build function to remove barcodes from experiments. This is done via the function `MPRAData.apply_barcode_filter()`. This function has pre-build filters like setting a minimum or maximum threshold for counts. E.g. you want to remove very very high barcode counts or you 'trust' only barcode counts with 3 RNA counts to remove noisy data. The functin can also randomly remove barcodes if you want to downsample your data on barcodes (Which correspons to removing barcodes from your assignment, theerfor ehaving a lower quality/lower depth assignment file).

But it can also detect outlier barcodes. E.g. if you want to remove barcodes that do not follow the distribution per oligo. This is usually done per repliacte.

Now we try to remove barcodes that are outliers within the RNA counts using the zscore:

In [17]:
from mpralib.mpradata import BarcodeFilter

# using standard number of barcodes per oligo
mpra_barcode_data.barcode_threshold = 10

# Resetting the barcode filter
mpra_barcode_data.barcode_filter = None
mpra_oligo_data = mpra_barcode_data.oligo_data
print(f"Mean Pearson correlation before BC filter, BC threshold {mpra_oligo_data.barcode_threshold}")
print(mpra_oligo_data.correlation().flatten()[[1,2,5]].mean().round(3))
print(f"Number of oligs")
print(np.sum(mpra_oligo_data.barcode_counts >= mpra_oligo_data.barcode_threshold,axis=1))

# Apply filter
mpra_barcode_data.apply_barcode_filter(BarcodeFilter.RNA_ZSCORE, {"times_zscore": 3})

mpra_oligo_data = mpra_barcode_data.oligo_data
print(f"Mean Pearson correlation after BC filter, BC threshold {mpra_oligo_data.barcode_threshold}")
print(mpra_oligo_data.correlation().flatten()[[1,2,5]].mean().round(3))
print(f"Number of oligs")
print(np.sum(mpra_oligo_data.barcode_counts >= mpra_oligo_data.barcode_threshold,axis=1))

Mean Pearson correlation before BC filter, BC threshold 10
0.629
Number of oligs
[1359 1349 1357]
Mean Pearson correlation after BC filter, BC threshold 10
0.602
Number of oligs
[1359 1349 1357]


You see we keeping number of oligos the same but our Person correlaton drops. Maybe within this example dataset this was not a good idea. But we want to see the barcodes that were removed. We can access them using the  `MPRAData.barcode_filter` matrix on the `MPRAData.var` (our barcodes) object. We also compute all barcodes that are consistently renoved across repliactes.

In [18]:
common_barcodes = None
for i, replicate in enumerate(mpra_barcode_data.var_filter):
    print(f"Number of removed barcodes in replicate {i}: {len(mpra_barcode_data.data.var_names[mpra_barcode_data.var_filter.iloc[:,i]])}")
    if common_barcodes is None:
        common_barcodes = mpra_barcode_data.data.var_names[mpra_barcode_data.var_filter.iloc[:,i]]
    else:
        common_barcodes = common_barcodes.intersection(mpra_barcode_data.data.var_names[mpra_barcode_data.var_filter.iloc[:,i]])
print(f"Number of common barcodes: {len(common_barcodes)}")
print(f"Common barcodes: {common_barcodes}")

Number of removed barcodes in replicate 0: 7389
Number of removed barcodes in replicate 1: 6349
Number of removed barcodes in replicate 2: 6614
Number of common barcodes: 29
Common barcodes: Index(['CCACGCCTGTCCAAC', 'TCACTGAACTAGTGC', 'GCTATCCACCGTAAC',
       'GAAGACTTTGAATAC', 'AACACGTACCTCACC', 'CTCATATCCGACGGC',
       'TCACCAGTTCTGCTG', 'TCCTTACTGTTGGGT', 'CTTTGACGTATCACC',
       'ACAATCTCACAGCTG', 'GCCACTTCGGCTAAA', 'CTCCTCGTAAGTTGA',
       'CCTGACAACGCAAAT', 'AAACAGGTCCGAGCG', 'TGAATTCCGGCGCCC',
       'GATTCAAAAGCGTCC', 'ACACACAAAAGCCTA', 'AAACCGGCCATAATA',
       'TCACCCCTCTCCCAC', 'CCTTCCGAGCGCAGC', 'AATGTATGAGAACGC',
       'TACGCCTCGCAACTG', 'GGAACGACTCAAGCC', 'GACTCCGTCTACTCC',
       'CCTAACTTCCCACCG', 'CTGGCTTGGGGCGAG', 'AGCCCGGGGGACAGC',
       'CACCTTCGCCATTTG', 'CTAGCTATGCGCTAA'],
      dtype='object', name='barcode')


Let's see if we achieve something else when allowing only a higher number of DNA and RNA counts:

In [19]:
mpra_barcode_data.barcode_filter = None

mpra_oligo_data = mpra_barcode_data.oligo_data
print(f"Mean Pearson correlation before BC filter, BC threshold {mpra_oligo_data.barcode_threshold}")
print(mpra_oligo_data.correlation().flatten()[[1,2,5]].mean().round(3))
print(f"Number of oligs")
print(np.sum(mpra_oligo_data.barcode_counts >= mpra_oligo_data.barcode_threshold,axis=1))

# Apply filter
mpra_barcode_data.apply_barcode_filter(BarcodeFilter.MIN_COUNT, {"dna_min_count": 2, "rna_min_count": 4})

mpra_oligo_data = mpra_barcode_data.oligo_data
print(f"Mean Pearson correlation before BC filter, BC threshold {mpra_oligo_data.barcode_threshold}")
print(mpra_oligo_data.correlation().flatten()[[1,2,5]].mean().round(3))
print(f"Number of oligs")
print(np.sum(mpra_oligo_data.barcode_counts >= mpra_oligo_data.barcode_threshold,axis=1))

Mean Pearson correlation before BC filter, BC threshold 10
0.602
Number of oligs
[1359 1349 1357]
Mean Pearson correlation before BC filter, BC threshold 10
0.852
Number of oligs
[10  5  7]


Now we see an improved correlaton. But hold on... We have nearly no oligo left that fullfills the BC threshold 10 criteria. This example data seems to be a very very low count data. Maybe not the best idea. 

## Saving activity files.

Finally we want to safe our oligo activities. We have a pre-build function for that.

In [20]:
from mpralib.utils import export_activity_file, export_barcode_file
export_activity_file(mpra_oligo_data, "activity.tsv")

df = pd.read_csv("activity.tsv", sep='\t')
display(df)

,replicate,oligo_name,dna_counts,rna_counts,dna_normalized,rna_normalized,log2FoldChange,n_bc
0,1,A:HNF4A-ChMod_chr5:78281698-78281840__chr5:782...,71,154,1160.7386,1421.1795,0.2920,27
1,1,A:HNF4A-NoMod_chr17:37895425-37895596__chr17:3...,45,92,1219.2197,1430.9943,0.2311,16
2,1,C:SLEA_hg18:chr9:82902419-82902586|13:V_HNF6_Q...,40,100,1172.5829,1625.3268,0.4710,15
3,1,R:EP300-ChMod_chr19:3397595-3397766__chr19:339...,32,69,1106.9838,1337.2254,0.2726,13
4,1,R:EP300-ChMod_chr4:6784271-6784442__chr4:67842...,39,69,1210.6538,1256.8521,0.0540,14
5,1,R:EP300-NoMod_chr3:23958571-23958742__chr3:239...,27,47,1104.7475,1117.8137,0.0170,11
6,1,R:FOXA1_FOXA2-ChMod_chr5:74172389-74172557__ch...,57,79,1535.0176,1328.5280,-0.2084,15
7,1,R:FOXA1_FOXA2-ChMod_chr6:11393329-11393497__ch...,24,44,1087.3041,1144.7954,0.0743,10
8,1,R:HNF4A-ChMod_chr1:27320293-27320362__chr1:273...,46,67,1370.5514,1226.5665,-0.1601,14
9,1,R:HNF4A-NoMod_chr5:42824710-42824777__chr5:428...,46,86,1239.2069,1351.4946,0.1251,16


So every oligo per replicate is written into one row with its counts, normalized counts, log2 fold change and the number of supporting barcodes. In total there are only 25 entries beacuse we did this drastical filtering before.

We can also write out the results in a barcode format

In [21]:
export_barcode_file(mpra_barcode_data, "barcodes.tsv")

df = pd.read_csv("barcodes.tsv", sep='\t')
display(df)

,barcode,oligo_name,dna_count_1,rna_count_1,dna_count_2,rna_count_2,dna_count_3,rna_count_3
0,TACTCTCCGTGCCCA,A:HNF4A-ChMod_chr10:11917871-11917984__chr10:1...,NaN,NaN,NaN,NaN,NaN,NaN
1,GGTATAACATCTCCG,A:HNF4A-ChMod_chr10:11917871-11917984__chr10:1...,NaN,NaN,NaN,NaN,NaN,NaN
2,TTAGGAGTCACACGT,A:HNF4A-ChMod_chr10:11917871-11917984__chr10:1...,NaN,NaN,NaN,NaN,NaN,NaN
3,GAATATAACACCCGA,A:HNF4A-ChMod_chr10:11917871-11917984__chr10:1...,NaN,NaN,NaN,NaN,NaN,NaN
4,AAACACCGCGCTCTA,A:HNF4A-ChMod_chr10:11917871-11917984__chr10:1...,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
92724,ACCCTATCAACGGCT,R:HNF4A-NoMod_chrY:18213828-18213963__chrY:182...,NaN,NaN,NaN,NaN,NaN,NaN
92725,AAGCGCTCGGCGATT,R:HNF4A-NoMod_chrY:18213828-18213963__chrY:182...,NaN,NaN,NaN,NaN,NaN,NaN
92726,CCCGACGGTCCCGGC,R:HNF4A-NoMod_chrY:18213828-18213963__chrY:182...,NaN,NaN,NaN,NaN,NaN,NaN
92727,CCAATCATCGAACTT,R:HNF4A-NoMod_chrY:18213828-18213963__chrY:182...,NaN,NaN,NaN,NaN,NaN,NaN


This dumped the whole DNA and rna counts into a file. Same format as we loaded the data. BUT now with the filtereing applied. So we see a lot of missing values.